<a href="https://colab.research.google.com/github/raminbazr/Python/blob/main/Classification_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel(r"/content/SampleData_Eval.xlsx")

In [ ]:
df.head()

,Row,Drug 1 (mg),Drug 2 (mg),Main Disease Status,Family Size,Age,Severity of the Relevant Disease,Surgery,Smoking Status,Pack-year,Diet Type,Gender,Race
0,0,143.266667,154.600000,1,1,50,4,No Surgery,Former,30.0,B,Male,White/Caucasian
1,1,126.666667,157.033333,1,2,56,4,No Surgery,Never,0.0,B,Male,NaN
2,2,109.333333,118.266667,1,3,62,4,Site 1,Former,0.0,A,Male,NaN
3,3,104.533333,113.266667,1,2,66,4,No Surgery,Never,0.0,B,Male,White/Caucasian
4,4,95.700000,109.666667,1,2,61,3,Site 2,Never,0.0,B,Male,White/Caucasian


In [ ]:
df["Gender"].unique()

array(['Male', 'Female'], dtype=object)

In [ ]:
df["Surgery"].unique()

array(['No Surgery', 'Site 1', 'Site 2', 'Site 4', 'Site 3'], dtype=object)

In [ ]:
df["Smoking Status"].unique()

array(['Former', 'Never', 'Current'], dtype=object)

In [ ]:
df["Diet Type"].unique()

array(['B', 'A', 'D', 'C', 'E'], dtype=object)

In [ ]:
df["Race"].unique()

array(['White/Caucasian', nan, 'Asian', 'Black or African American',
       'Other', 'Hispanic/Latino', 'other', 'African American/Black'],
      dtype=object)

In [ ]:
df["Race"]= df["Race"].str.replace ('Black or African American','African American/Black')

In [ ]:
df["Race"]= df["Race"].str.replace ('other','Other')

In [ ]:
df["Race"].unique()

array(['White/Caucasian', nan, 'Asian', 'African American/Black', 'Other',
       'Hispanic/Latino'], dtype=object)

In [ ]:
df["Race"].value_counts()

White/Caucasian           293
Hispanic/Latino            19
African American/Black     12
Asian                       5
Other                       3
Name: Race, dtype: int64

In [ ]:
#because of most of Race Column values are "White/Caucasian", we replace it to the nan cells.
df.Race.fillna(value="White/Caucasian", inplace=True)

In [ ]:
#working with columns name that have space in their name is hard. So, we rename them.
df = df.rename(columns={'Drug 1 (mg)': 'Drug_one'})
df = df.rename(columns={'Drug 2 (mg)': 'Drug_two'})

In [ ]:
#if we see the trend of data in Drug_one column, they are descending. Given that we select 0 for blank cells.
df.Drug_one.fillna(value=0, inplace=True)

In [ ]:
df.count()

Row                                  399
Drug_one                             399
Drug_two                             398
Main Disease Status                  399
Family Size                          399
Age                                  399
Severity of the Relevant Disease     399
Surgery                              399
Smoking Status                       399
Pack-year                            399
Diet Type                            399
Gender                               399
Race                                 399
dtype: int64

In [ ]:
# we found that in Drug_2 column there is a blank column. In this regard, we put mean in the blank cell.
df.Drug_two.fillna(value=df["Drug_two"].mean(), inplace=True)

In [ ]:
df.count()

Row                                  399
Drug_one                             399
Drug_two                             399
Main Disease Status                  399
Family Size                          399
Age                                  399
Severity of the Relevant Disease     399
Surgery                              399
Smoking Status                       399
Pack-year                            399
Diet Type                            399
Gender                               399
Race                                 399
dtype: int64

In [ ]:
#people who ages are less than 1 cannot be alive to use drugs or smoke or diet
df = df.drop(df[df["Age"]== 0].index)

In [ ]:
#I have no idea about people under 18 years old can use smoking or not
#However, I suppose that they cannot use smoke (in former or current mode), and these are false records.
df = df.drop (df[(df["Age"]<= 18) & (df["Smoking Status"]=='Former')].index)
df = df.drop (df[(df["Age"]<= 18) & (df["Smoking Status"]=='Current')].index)

In [ ]:
#when we have pack-year = 0, we should have Smoking Style = 'never'
#but, I see in some cells when Pack-year=0, the Smoking Style = 'Former'
#because of this, I change their name to 'never'
df.loc [df['Pack-year']==0 , 'Smoking Status' ]= 'Never'

In [ ]:
df.count()

Row                                  372
Drug_one                             372
Drug_two                             372
Main Disease Status                  372
Family Size                          372
Age                                  372
Severity of the Relevant Disease     372
Surgery                              372
Smoking Status                       372
Pack-year                            372
Diet Type                            372
Gender                               372
Race                                 372
dtype: int64

In [ ]:
#Row column shows the index, and it's not necessary
df = df.drop(columns=["Row"])

In [ ]:
#because we remove some columns, we should reset the indexes.
df = df.reset_index(drop='true')

**Convert Data to Numeric**

In [ ]:
#for gender Column, we use one-hot encoding to convert them to two numeric columns
df = pd.get_dummies(df, columns = ["Gender"])

In [ ]:
# for others, we cannot use one-hot encoding because we generate many columns.
#in this regard, we use lable encoding
from sklearn import preprocessing
lable_encoder = preprocessing.LabelEncoder()

df["Surgery"]= lable_encoder.fit_transform(df["Surgery"])
df["Smoking Status"] = lable_encoder.fit_transform (df["Smoking Status"])
df["Diet Type"]= lable_encoder.fit_transform (df["Diet Type"])
df["Race"]= lable_encoder.fit_transform(df["Race"])

In [ ]:
#Now, all of our columns are Numeric
df.head()

,Drug_one,Drug_two,Main Disease Status,Family Size,Age,Severity of the Relevant Disease,Surgery,Smoking Status,Pack-year,Diet Type,Race,Gender_Female,Gender_Male
0,143.266667,154.600000,1,1,50,4,0,1,30.0,1,4,0,1
1,126.666667,157.033333,1,2,56,4,0,2,0.0,1,4,0,1
2,109.333333,118.266667,1,3,62,4,1,2,0.0,0,4,0,1
3,104.533333,113.266667,1,2,66,4,0,2,0.0,1,4,0,1
4,95.700000,109.666667,1,2,61,3,2,2,0.0,1,4,0,1


# **Classification Model**

**Binary Classification**

In [ ]:
# Import libraries and classes required
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Assign values to the X and y variables:
X = df.drop("Main Disease Status", axis=1)
y = df["Main Disease Status"]
# Split dataset into random train and test subsets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
# Standardize features by removing mean and scaling to unit variance:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**KNN classifier to fit data**

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=5)
KNN.fit(X_train, y_train)
# Predict y data with classifier:
y_predict = KNN.predict(X_test)
# Print results:
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

[[46  1]
 [ 6 22]]
              precision    recall  f1-score   support

           0       0.88      0.98      0.93        47
           1       0.96      0.79      0.86        28

    accuracy                           0.91        75
   macro avg       0.92      0.88      0.90        75
weighted avg       0.91      0.91      0.90        75



**SVM Classifier**

In [ ]:
SVM = LinearSVC()
SVM.fit(X_train, y_train)
# Predict y data with classifier:
y_predict = SVM.predict(X_test)
# Print results:
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

[[47  0]
 [ 0 28]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        47
           1       1.00      1.00      1.00        28

    accuracy                           1.00        75
   macro avg       1.00      1.00      1.00        75
weighted avg       1.00      1.00      1.00        75



**RandomForest Classifier**

In [ ]:
RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)
# Predict y data with classifier:
y_predict = RFC.predict(X_test)
# Print results:
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

[[47  0]
 [ 1 27]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        47
           1       1.00      0.96      0.98        28

    accuracy                           0.99        75
   macro avg       0.99      0.98      0.99        75
weighted avg       0.99      0.99      0.99        75



# **Validation**

In [ ]:
#Because to each Model Run, we get different answers. I decided to validate the models.
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
kf =KFold(n_splits=3, shuffle=True, random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 248, Test set:124
Fold:2, Train set: 248, Test set:124
Fold:3, Train set: 248, Test set:124


In [ ]:
score = cross_val_score(KNN, X, y, cv= kf, scoring="accuracy")
print(f'Scores for each fold: {score}')
print(score.mean())

Scores for each fold: [0.83870968 0.81451613 0.88709677]
0.8467741935483871


In [ ]:
score = cross_val_score(SVM, X, y, cv= kf, scoring="accuracy")
print(f'Scores for each fold: {score}')
print(score.mean())

Scores for each fold: [1.         1.         0.98387097]
0.9946236559139785


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
score = cross_val_score(RFC, X, y, cv= kf, scoring="accuracy")
print(f'Scores for each fold: {score}')
print(score.mean())

Scores for each fold: [0.99193548 1.         1.        ]
0.9973118279569894
